In [1]:
#it's incomplete as it took a lot of time for my machine

import numpy as npxx
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [2]:
# Preprocess data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

In [3]:
# Define RBM class
class RBM:
    def __init__(self, visible_dim, hidden_dim):
        self.visible_dim = visible_dim
        self.hidden_dim = hidden_dim
        self.W = np.random.normal(0, 0.1, (visible_dim, hidden_dim))
        self.b_visible = np.zeros(visible_dim)
        self.b_hidden = np.zeros(hidden_dim)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sample_hidden(self, visible):
        p_hidden = self.sigmoid(np.dot(visible, self.W) + self.b_hidden)
        return np.random.binomial(1, p_hidden)

    def sample_visible(self, hidden):
        p_visible = self.sigmoid(np.dot(hidden, self.W.T) + self.b_visible)
        return np.random.binomial(1, p_visible)

    def contrastive_divergence(self, visible):
        hidden = self.sample_hidden(visible)
        visible_recon = self.sample_visible(hidden)
        hidden_recon = self.sample_hidden(visible_recon)

        positive = np.dot(visible.T, hidden)
        negative = np.dot(visible_recon.T, hidden_recon)

        self.W += 0.1 * (positive - negative)
        self.b_visible += 0.1 * np.mean(visible - visible_recon, axis=0)
        self.b_hidden += 0.1 * np.mean(hidden - hidden_recon, axis=0)

In [4]:
# Define CDBN model
cdbn = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
])


In [5]:
# RBM pretraining
cdbn.compile(optimizer='adam', loss='mse')

In [ ]:
num_epochs = 10x
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for i in range(len(x_train)):
        cdbn.fit(x_train[i:i+1], x_train[i:i+1], verbose=0)

Epoch 1/10


In [ ]:
# Predict labels for 5 images
predictions = cdbn.predict(x_test[:5])

In [ ]:
# Display the images and predicted labels
import matplotlib.pyplot as plt

for i in range(5):
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title(f'Predicted Label: {np.argmax(predictions[i])}')
    plt.axis('off')
    plt.show()